In [2]:
import dlib
import cv2
import numpy as np
import os
import time  # 실행 시간 측정을 위한 모듈

# 얼굴 감지기 및 얼굴 인코더 모델 로드
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("C:/face/shape_predictor_68_face_landmarks.dat")
face_rec_model = dlib.face_recognition_model_v1("C:/face/dlib_face_recognition_resnet_model_v1.dat")

# myface 폴더에서 기준 얼굴 이미지 로드
face_encodings = []
face_names = []
myface_dir = "C:/myface"

for file in os.listdir(myface_dir):
    img_path = os.path.join(myface_dir, file)
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    faces = detector(img_rgb)
    
    if len(faces) > 0:
        shape = predictor(img_rgb, faces[0])
        face_descriptor = face_rec_model.compute_face_descriptor(img_rgb, shape)
        face_encodings.append(np.array(face_descriptor))
        face_names.append(file.split('.')[0])  # 파일명을 이름으로 저장

print(f"✅ Loaded {len(face_encodings)} face encodings from {myface_dir}")

# 실시간 웹캠 얼굴 인식
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    start_time = time.time()  # 프레임 처리 시작 시간

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = detector(img_rgb)

    for face in faces:
        face_start_time = time.time()  # 얼굴 감지부터 비교까지의 시간 측정 시작

        shape = predictor(img_rgb, face)
        embedding_start_time = time.time()  # 얼굴 임베딩 시작 시간
        face_descriptor = face_rec_model.compute_face_descriptor(img_rgb, shape)
        embedding_time = (time.time() - embedding_start_time) * 1000  # ms 단위 변환
        face_encoding = np.array(face_descriptor)

        # 유사도 비교
        min_dist = float("inf")
        identity = "Unknown"

        comparison_start_time = time.time()  # 유사도 비교 시작 시간
        for stored_encoding, name in zip(face_encodings, face_names):
            dist = np.linalg.norm(stored_encoding - face_encoding)
            if dist < min_dist:
                min_dist = dist
                identity = name
        comparison_time = (time.time() - comparison_start_time) * 1000  # ms 단위 변환

        # 유사도 출력
        similarity = 1 - (min_dist / 1.0)  # 정규화 (거리가 0이면 1, 클수록 0)

        # 얼굴 박스 그리기
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        text = f"{identity}: {similarity:.2f}"
        cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        face_time = (time.time() - face_start_time) * 1000  # 얼굴 감지부터 비교까지 총 시간 (ms)

        # 실행 시간 정보 화면에 출력
        cv2.putText(frame, f"Embed: {embedding_time:.1f}ms", (x, y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
        cv2.putText(frame, f"Compare: {comparison_time:.1f}ms", (x, y + 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
        cv2.putText(frame, f"Total: {face_time:.1f}ms", (x, y + 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

    frame_time = (time.time() - start_time) * 1000  # 전체 프레임 처리 시간 (ms)
    print(f"프레임 처리 시간: {frame_time:.2f}ms")

    # 프레임 처리 시간 화면에 출력
    cv2.putText(frame, f"Frame Time: {frame_time:.1f}ms", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


✅ Loaded 1 face encodings from C:/myface
프레임 처리 시간: 528.05ms
프레임 처리 시간: 524.21ms
프레임 처리 시간: 496.45ms
프레임 처리 시간: 471.06ms
프레임 처리 시간: 482.11ms
프레임 처리 시간: 477.02ms
프레임 처리 시간: 470.90ms
프레임 처리 시간: 461.21ms
프레임 처리 시간: 462.93ms
프레임 처리 시간: 510.08ms
프레임 처리 시간: 465.59ms
프레임 처리 시간: 461.75ms
프레임 처리 시간: 463.26ms
프레임 처리 시간: 493.69ms
프레임 처리 시간: 466.37ms
프레임 처리 시간: 475.35ms
프레임 처리 시간: 458.45ms
프레임 처리 시간: 476.74ms
프레임 처리 시간: 484.65ms
프레임 처리 시간: 470.66ms
프레임 처리 시간: 466.10ms
프레임 처리 시간: 490.50ms
프레임 처리 시간: 486.78ms
프레임 처리 시간: 499.80ms
프레임 처리 시간: 471.12ms
프레임 처리 시간: 480.12ms
프레임 처리 시간: 485.31ms
프레임 처리 시간: 471.50ms
